# Dressmaker - Hard
You may need to create views to complete these questions - but you do not have permission to create tables or views in the default schema. Your SQL commands are executed by user scott in schema gisq - you may create or drop views and tables in schema scott but not in gisq.

In [1]:
# Prerequesites
from pyhive import hive
%load_ext sql
%sql hive://cloudera@quickstart.cloudera:10000/sqlzoo
%config SqlMagic.displaylimit = 20

## 1.
When creating a view in scott you must specify the schema name of the sources and the destination.

In [2]:
%%sql
CREATE SCHEMA IF NOT EXISTS scott

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


[]

## 2.
It is decided to review the materials stock. How much did each material contribute to turnover in 2002?

In [3]:
%%sql
SELECT material_no, fabric, colour, pattern, 
    ROUND(SUM(quantity), 2) AS quantity,
    ROUND(SUM(cost*quantity), 2) AS cost
    FROM material JOIN order_line ON (
        order_line.ol_material=material.material_no) JOIN quantities ON (
        quantities.size_q=order_line.ol_size AND
            quantities.style_q=order_line.ol_style) JOIN
        dress_order ON (dress_order.order_no=order_line.order_ref)
        WHERE YEAR(order_date)=2002
        GROUP BY material_no, fabric, colour, pattern

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


material_no,fabric,colour,pattern,quantity,cost
1,Silk,Black,Plain,4.9,34.3
2,Silk,Red Abstract,Printed,9.3,93.0
3,Cotton,Yellow Stripe,Woven,5.7,17.1
4,Cotton,Green Stripe,Woven,2.2,6.6
5,Cotton,Black Dotted,Woven,6.4,19.2
6,Cotton,Red Stripe,Woven,2.2,6.6
7,Polyester,Pale Yellow,Printed,4.3,10.96
8,Cotton,Blue Stripe,Woven,4.2,12.6
9,Cotton,Pink Check,Woven,4.6,13.8
10,Silk,Green Abstract,Printed,8.3,124.5


## 3.
An order for shorts has just been placed and the work is to be distributed amongst the workforce, and we wish to know how busy the shorts makers are. For each of the workers who have experience of making shorts show the number of hours work that she is currently committed to, assuming a meagre wage of £4.50 per hour

In [4]:
%%sql
WITH t AS (
    SELECT DISTINCT d_no
      FROM dressmaker JOIN construction ON (
          dressmaker.d_no=construction.maker) JOIN
          dress_order ON (dress_order.order_no=construction.order_ref) JOIN
          order_line ON (order_line.order_ref=dress_order.order_no) JOIN
        garment ON (garment.style_no=order_line.ol_style)
        WHERE LOWER(garment.description)='shorts'
)
SELECT d_name, ROUND(SUM(labour_cost)/4.5, 2) hrs
  FROM dressmaker JOIN construction ON (
      dressmaker.d_no=construction.maker) JOIN
      dress_order ON (dress_order.order_no=construction.order_ref) JOIN
        order_line ON (order_line.order_ref=dress_order.order_no) JOIN
        garment ON (garment.style_no=order_line.ol_style) JOIN
        quantities ON (quantities.style_q=order_line.ol_style AND
                       quantities.size_q=order_line.ol_size) JOIN
        t ON (dressmaker.d_no=t.d_no)
    WHERE completed='N'
    GROUP BY d_name

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


d_name,hrs
Miss Pins,28.17
Miss Stitch,49.17
Mr Needles,18.61
Mr Seam,28.17
Mr Taylor,18.61
Ms Sew,18.61


## 4.
"Big spender of the year" is the customer who spends the most on high value items. Identify the "Big spender of the year 2002" if the "high value" threshold is set at £30. Also who would it be if the threshold was £20 or £50?

In [5]:
%%sql
WITH t AS (
  SELECT order_ref, line_no, quantity*cost+labour_cost tot_cost
    FROM order_line JOIN quantities ON (
        order_line.ol_style=quantities.style_q AND 
        order_line.ol_size=quantities.size_q) JOIN
    garment ON (order_line.ol_style=garment.style_no) JOIN
    material ON (material.material_no=order_line.ol_material)
)
SELECT c_name, thres, ROUND(cost, 2) spending FROM
(SELECT c_name, 20 AS thres, SUM(tot_cost) AS cost
  FROM (SELECT * FROM t WHERE tot_cost>=20) t JOIN dress_order ON (
      t.order_ref=dress_order.order_no) JOIN
    jmcust ON (jmcust.c_no=dress_order.cust_no)
    GROUP BY c_name
    ORDER BY cost DESC
    LIMIT 1
UNION ALL
SELECT c_name, 30 AS thres, SUM(tot_cost) AS cost
  FROM (SELECT * FROM t WHERE tot_cost>=30) t JOIN dress_order ON (
      t.order_ref=dress_order.order_no) JOIN
    jmcust ON (jmcust.c_no=dress_order.cust_no)
    GROUP BY c_name
    ORDER BY cost DESC
    LIMIT 1
UNION ALL
SELECT c_name, 50 AS thres, SUM(tot_cost) AS cost
  FROM (SELECT * FROM t WHERE tot_cost>=50) t JOIN dress_order ON (
      t.order_ref=dress_order.order_no) JOIN
    jmcust ON (jmcust.c_no=dress_order.cust_no)
    GROUP BY c_name
    ORDER BY cost DESC
    LIMIT 1) a

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


c_name,thres,spending
Mr Brass,20,198.54
Ms White,30,173.55
Mr Brass,50,72.0


## 5.
Who is the fastest at making trousers?

In [6]:
%%sql
SELECT d_no, d_name, DATEDIFF(finish_date, start_date) days
  FROM dressmaker JOIN construction ON (
      dressmaker.d_no=construction.maker) JOIN
      order_line ON (order_line.order_ref=construction.order_ref AND
                order_line.line_no=construction.line_ref) JOIN
    garment ON (garment.style_no=order_line.ol_style)
    WHERE LOWER(description)='trousers' AND
        finish_date IS NOT NULL AND start_date IS NOT NULL
    ORDER BY days
    LIMIT 1

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


d_no,d_name,days
3,Mr Needles,3


## 6.
"Employee of the month" is the seamstress who completes the greatest value of clothes. Show the "employees of the month" for months in 2002.

In [7]:
%%sql
WITH t AS (
  SELECT d_name, MONTH(start_date) AS month, 
    ROUND(SUM(quantity*cost+labour_cost), 2) val,
    RANK() OVER(PARTITION BY MONTH(start_date) 
                ORDER BY SUM(quantity*cost+labour_cost) DESC) AS rank
    FROM material JOIN order_line ON (
        order_line.ol_material=material.material_no) JOIN quantities ON (
        quantities.size_q=order_line.ol_size AND
            quantities.style_q=order_line.ol_style) JOIN garment ON (
        garment.style_no=order_line.ol_style) JOIN construction ON (
        order_line.order_ref=construction.order_ref AND
            order_line.line_no=construction.line_ref) JOIN dressmaker ON (
        dressmaker.d_no=construction.maker)
        WHERE YEAR(start_date)=2002
        GROUP BY d_name, MONTH(start_date)
)
SELECT month, d_name FROM t WHERE rank=1

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


month,d_name
1,Miss Stitch
2,Mrs Hem
3,Miss Stitch
